In [1]:
import tensorflow as tf
import scipy as sp
import numpy as np
import pandas as pd
import re
import hashlib
from sklearn.preprocessing import StandardScaler

import os
from pathlib import Path
import IPython.display as ipd
import cProfile

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
from scipy import signal
from scipy.io import wavfile
%matplotlib inline


from tensorflow.python.keras.callbacks import ModelCheckpoint

from tensorflow.python.keras.utils import to_categorical

from sklearn.preprocessing import OrdinalEncoder

from tensorflow.python.keras.models import Sequential

from tensorflow.python.keras.layers import Input, Dense, LSTM

from tensorflow.python.keras.layers import Embedding, Dropout, Activation

import sys
sys.path.insert(0, '../src/data')
import get_train_test

In [2]:
from tensorflow.python.keras.models import load_model

In [3]:
model = load_model('./model-31.hdf5')

In [4]:
raw_train, raw_dev = get_train_test.get_train_test('../data/raw/train/audio/',10,12)

1000 : 2.7230554761253236 s
2000 : 4.104234709429624 s
3000 : 5.46087198394748 s
4000 : 6.815128539539914 s
5000 : 8.154786525040626 s


In [5]:
x_train = np.array([np.pad(x[0], ((0,99 - x[0].shape[0]),(0,0)), 'constant', constant_values=(0)) for x in raw_train])
x_dev = np.array([np.pad(x[0], ((0,99 - x[0].shape[0]),(0,0)), 'constant', constant_values=(0)) for x in raw_dev])

y_train = np.array([x[1] for x in raw_train])
y_dev = np.array([x[1] for x in raw_dev])

In [6]:
x_resh = x_train.reshape(-1, 99*161)

scaler = StandardScaler()

scaler.fit(x_resh)


StandardScaler(copy=True, with_mean=True, with_std=True)

In [10]:
parts = 360

In [12]:
for i in range(0,parts):
    print (i)
    raw_test = get_train_test.get_test('../data/raw/test/audio/',parts,i)

    test_fnames = np.array([x[0] for x in raw_test])

    x_test = np.array([np.pad(x[1], ((0,99 - x[1].shape[0]),(0,0)), 'constant', constant_values=(0)) for x in raw_test])

    x_test_scaled = scaler.transform(x_test.reshape(-1,99*161)).reshape(-1,99,161)

    x_pred = model.predict(x_test_scaled)

    x_best_pred = np.argmax(x_pred,axis=1)

    labels = (OrdinalEncoder().fit(y_train.reshape(-1,1)).categories_[0])# [ 'down', 'go', 'left', 'no',  'off','on','right', 'stop','unknown', 'up', 'yes', ]

    f = lambda x: labels[x]
    f = np.vectorize(f)
    response = f(x_best_pred)

    total_response = np.concatenate([test_fnames.reshape(test_fnames.shape[0],-1),response.reshape(test_fnames.shape[0],-1)],axis = 1)

    f=open('resp.csv','ab')
    np.savetxt(f, total_response, delimiter=",", fmt='%s')
    f.close()

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27